In [4]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import joblib

FILE = "sarima_residuals.csv"
CATEGORY_COL = "second-level_category"

os.makedirs("lstm_models", exist_ok=True)

# Load residual dataset
df = pd.read_csv(FILE, parse_dates=["date"])
categories = df[CATEGORY_COL].unique()

def create_sequences(values, window=12):
    X, y = [], []
    for i in range(len(values) - window):
        X.append(values[i:i+window])
        y.append(values[i+window])
    return np.array(X), np.array(y)

WINDOW = 12

print("\n=== Training LSTM models per Category ===\n")
for cat in categories:
    data = df[df[CATEGORY_COL] == cat].sort_values("date")

    if len(data) < WINDOW + 6:
        continue

    res = data["residual"].values.reshape(-1, 1)

    scaler = MinMaxScaler(feature_range=(-1,1))
    res_scaled = scaler.fit_transform(res)

    X, y = create_sequences(res_scaled, WINDOW)
    if len(X) < 20:  
        continue

    train_split = int(len(X) * 0.8)
    X_train, X_test = X[:train_split], X[train_split:]
    y_train, y_test = y[:train_split], y[train_split:]

    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(WINDOW,1)),
        Dropout(0.2),
        LSTM(32),
        Dense(1)
    ])
    model.compile(optimizer="adam", loss="mse")

    es = EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True)

    model.fit(X_train, y_train,
              epochs=100, batch_size=16,
              validation_data=(X_test, y_test),
              callbacks=[es], verbose=0)

    model.save(f"lstm_models/lstm_{cat}.keras")
    joblib.dump(scaler, f"lstm_models/scaler_{cat}.pkl")

print("\n✨ LSTM Models Trained & Saved!")


=== Training LSTM models per Category ===



c:\Users\Ven\Documents\Coding\pambihira\CRAZY\poop\ShopeeAnalysis_Amalgam\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\Ven\Documents\Coding\pambihira\CRAZY\poop\ShopeeAnalysis_Amalgam\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\Ven\Documents\Coding\pambihira\CRAZY\poop\ShopeeAnalysis_Amalgam\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instea

KeyboardInterrupt: 

In [1]:
# --- Evaluation Metrics: RMSE, MAE, MAPE ---
from sklearn.metrics import mean_squared_error, mean_absolute_error

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    nonzero = y_true != 0
    return np.mean(np.abs((y_true[nonzero] - y_pred[nonzero]) / y_true[nonzero])) * 100 if np.any(nonzero) else np.nan

for cat in categories:
    data = df[df[CATEGORY_COL] == cat].sort_values("date")
    if len(data) < WINDOW + 6:
        continue
    res = data["residual"].values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(-1,1))
    res_scaled = scaler.fit_transform(res)
    X, y = create_sequences(res_scaled, WINDOW)
    if len(X) < 20:
        continue
    train_split = int(len(X) * 0.8)
    X_train, X_test = X[:train_split], X[train_split:]
    y_train, y_test = y[:train_split], y[train_split:]
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(WINDOW,1)),
        Dropout(0.2),
        LSTM(32),
        Dense(1)
    ])
    model.compile(optimizer="adam", loss="mse")
    es = EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True)
    model.fit(X_train, y_train,
              epochs=100, batch_size=16,
              validation_data=(X_test, y_test),
              callbacks=[es], verbose=0)
    # --- Evaluation ---
    y_pred = model.predict(X_test)
    y_test_inv = scaler.inverse_transform(y_test)
    y_pred_inv = scaler.inverse_transform(y_pred)
    rmse = mean_squared_error(y_test_inv, y_pred_inv, squared=False)
    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)
    print(f"Category: {cat}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  MAPE: {mape:.2f}%\n")
    model.save(f"lstm_models/lstm_{cat}.keras")
    joblib.dump(scaler, f"lstm_models/scaler_{cat}.pkl")

NameError: name 'categories' is not defined